# 제네릭
- Kotlin은 컬렉션 사용 시 제네릭 타입 지정을 강제함 (Java는 선택사항)
-  Kotlin의 컴파일러는 컨텍스트를 통해 제네릭 타입을 추론할 수 있어, 명시적 타입 선언을 줄일 수 있습니다.

### 제네릭의 장점

- 컴파일 타임에 타입 검사 가능
- 코드가 더 명확해짐
- 캐스팅 횟수 감소



In [3]:
val list: MutableList<String> = mutableListOf("hello")
val liet = mutableListOf<String>()
// val list = mutableListOf() // 오류 발생!

// 타입 추론 활용
val list2 = mutableListOf("hello") // 타입이 자동으로 MutableList<String>로 추론됨

In [4]:
import java.math.BigDecimal

fun <T> printCollection(collection: Collection<T>) {
    for (item in collection) {
        println(item)
    }
}

val stringList = mutableListOf("hello", "world")
val numberList = mutableListOf(BigDecimal("33.45"), BigDecimal("350.99"))

printCollection(stringList)
printCollection(numberList)


hello
world
33.45
350.99


### 확장 함수

In [5]:
fun <T> List<T>.printCollection() {
    for (item in this) {
        println(item)
    }
}

stringList.printCollection()
numberList.printCollection()

hello
world
33.45
350.99


### 타입 매개변수 선언
- <T>: 함수명 앞에 위치하는 타입 매개변수 선언
- T는 관례적으로 사용되는 이름으로 "any Type"을 의미
- 여러 타입 매개변수 사용 가능: <T, U, V>